In [6]:
import torch
import torch.nn as nn

# Definir la clase EmotionRecognitionModel antes de cargar el modelo
class EmotionRecognitionModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(EmotionRecognitionModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        h_0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        c_0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h_0, c_0))
        out = out[:, -1, :]  # Tomar la salida del último timestep
        out = self.fc(out)
        return out

# Cargar el modelo completo
model = torch.load('modelo_emociones_lstm_completo.pth')



/var/folders/pv/hj1tkdtn4xd2lk842kp9xy7r0000gn/T/ipykernel_6684/4264266241.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('modelo_emociones_lstm_com

In [ ]:
import torch
import numpy as np
from collections import deque

# Asegúrate de que el modelo esté en modo de evaluación y en el dispositivo correcto
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.eval()

# Configuración de la secuencia de tiempo (timesteps)
n_timesteps = 101  # Debe coincidir con lo que usaste para entrenar el modelo
n_features = 38  # Asegúrate de que esto coincida con el número de características por timestep usado en el entrenamiento

# Inicializar un buffer para mantener las secuencias de características en tiempo real
sequence_buffer = deque(maxlen=n_timesteps)

# Inicializar la emoción previa para detección de cambios
previous_emotion = None

# Simulación de obtención de características en tiempo real (ajusta para que devuelva 38 características)
def get_real_time_features():
    # Esta función debería devolver un vector de características de longitud `n_features` (38 en este caso)
    # Aquí simplemente devolvemos datos aleatorios como ejemplo
    return np.random.rand(n_features)

# Ejemplo de bucle de predicción en tiempo real
while True:
    # Obtener nuevas características en tiempo real
    new_features = get_real_time_features()

    # Añadir las nuevas características al buffer
    sequence_buffer.append(new_features)

    # Verificar si hemos llenado el buffer con suficientes timesteps
    if len(sequence_buffer) == n_timesteps:
        # Convertir el buffer a un tensor y hacer la predicción
        sequence_array = np.array(sequence_buffer)
        input_tensor = torch.tensor(sequence_array, dtype=torch.float32).unsqueeze(0).to(device)  # Añadir dimensión de batch

        with torch.no_grad():
            output = model(input_tensor)
            _, predicted = torch.max(output, 1)
            current_emotion = predicted.item()

        # Detectar cambio de emoción
        if current_emotion != previous_emotion:
            print(f'Emoción cambiada: {current_emotion}')
            previous_emotion = current_emotion
            # Aquí podrías agregar lógica para mostrar la nueva emoción, guardarla, etc.

    # Simulación de esperar un poco antes de la siguiente iteración (ej. 100ms)
    # Para tiempos reales, esta parte dependería de cómo obtienes las características en tiempo real
    # time.sleep(0.1)


In [8]:
import torch
import numpy as np
from collections import deque, Counter

# Asegúrate de que el modelo esté en modo de evaluación y en el dispositivo correcto
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.eval()

# Configuración de la secuencia de tiempo (timesteps)
n_timesteps = 101
n_features = 38

# Inicializar un buffer para mantener las secuencias de características en tiempo real
sequence_buffer = deque(maxlen=n_timesteps)

# Ventana deslizante para suavizar predicciones
prediction_window = deque(maxlen=5)  # Ventana de las últimas 5 predicciones

# Inicializar la emoción previa para detección de cambios
previous_emotion = None

# Simulación de obtención de características en tiempo real
def get_real_time_features():
    return np.random.rand(n_features)

# Bucle de predicción en tiempo real
while True:
    new_features = get_real_time_features()
    sequence_buffer.append(new_features)

    if len(sequence_buffer) == n_timesteps:
        sequence_array = np.array(sequence_buffer)
        input_tensor = torch.tensor(sequence_array, dtype=torch.float32).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            _, predicted = torch.max(output, 1)
            current_emotion = predicted.item()

        # Añadir la nueva predicción a la ventana deslizante
        prediction_window.append(current_emotion)

        # Solo cambiar la emoción si la mayoría de las últimas predicciones son consistentes
        most_common_emotion, count = Counter(prediction_window).most_common(1)[0]
        if most_common_emotion != previous_emotion and count > 2:  # Cambiar solo si hay al menos 3 predicciones consistentes
            print(f'Emoción cambiada: {most_common_emotion}')
            previous_emotion = most_common_emotion

    # time.sleep(0.1)  # Ajusta según la velocidad de adquisición de características en tiempo real


Emoción cambiada: 6
Emoción cambiada: 4
Emoción cambiada: 6
Emoción cambiada: 4
Emoción cambiada: 6
Emoción cambiada: 4
Emoción cambiada: 6
Emoción cambiada: 4
Emoción cambiada: 6
Emoción cambiada: 4
Emoción cambiada: 6
Emoción cambiada: 4
Emoción cambiada: 6
Emoción cambiada: 4
Emoción cambiada: 6
Emoción cambiada: 4
Emoción cambiada: 6
Emoción cambiada: 4
Emoción cambiada: 6
Emoción cambiada: 4
Emoción cambiada: 6
Emoción cambiada: 4
Emoción cambiada: 6
Emoción cambiada: 4


KeyboardInterrupt: 

In [ ]:
import torch
import numpy as np
from collections import deque

# Cargar el modelo completo
model = torch.load('modelo_emociones_lstm_completo.pth')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()  # Establecer el modelo en modo de evaluación

# Configuración de la secuencia de tiempo (timesteps)
n_timesteps = 101  # Asegúrate de que coincida con lo que espera tu modelo
n_features = model.fc.in_features  # Esto debería coincidir con el número de características por timestep

# Inicializar un buffer para mantener las secuencias de características de tiempo real
sequence_buffer = deque(maxlen=n_timesteps)

# Suponiendo que tienes un flujo de características en tiempo real (aquí usaremos una función ficticia)
def get_real_time_features():
    # Esta función debería devolver un vector de características de longitud `n_features`
    # Aquí simplemente devolveremos datos aleatorios como ejemplo
    return np.random.rand(n_features)

# Ejemplo de bucle de predicción en tiempo real
while True:
    # Obtener nuevas características en tiempo real
    new_features = get_real_time_features()

    # Añadir las nuevas características al buffer
    sequence_buffer.append(new_features)

    # Verificar si hemos llenado el buffer con suficientes timesteps
    if len(sequence_buffer) == n_timesteps:
        # Convertir el buffer a un tensor y hacer la predicción
        sequence_array = np.array(sequence_buffer)
        input_tensor = torch.tensor(sequence_array, dtype=torch.float32).unsqueeze(0).to(device)  # Añadir dimensión de batch

        with torch.no_grad():
            output = model(input_tensor)
            _, predicted = torch.max(output, 1)
            predicted_emotion = predicted.item()
            print(f'Emoción predicha: {predicted_emotion}')

        # Aquí podrías agregar una lógica para mostrar la emoción en pantalla, guardarla, etc.

    # Simulación de esperar un poco antes de la siguiente iteración (ej. 100ms)
    # Para tiempos reales, esta parte dependería de cómo obtienes las características en tiempo real
    # time.sleep(0.1)
